<a href="https://colab.research.google.com/github/Kaiziferr/multiple_models/blob/main/OneVsRest_OneVsOne/2_OneVsOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import warnings
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

**Info**
---
**@By:** Steven Bernal

**@Nickname:** Kaiziferr

**@Git:** https://github.com/Kaiziferr

The goal of this exercise is to implement the One-vs-One strategy for a classification problem.

°°°°

El objetivo de este ejercicio es implementar la estrategia Uno contra Uno para un problema de clasificación.

# **Config**
---

In [42]:
pd.set_option('display.float_format', '{:,.5f}'.format)
random_seed=73
warnings.filterwarnings("ignore")

# **Data**

---



A dataset of mining traffic will be used, without going into too many details.

°°°°

Se utilizará un conjunto de datos de tráfico minero, sin entrar en demasiados detalles.

In [43]:
url = 'https://raw.githubusercontent.com/Kaiziferr/datasets/main/cryptojacking.csv'
data = pd.read_csv(url)

In [44]:
data['Type'].value_counts().reset_index().T

,0,1,2,3,4,5
Type,3,0,1,5,2,4
count,950,696,485,267,247,192


Three types of cryptocurrencies were selected to make the exercise more practical.

°°°°

Se seleccionaron tres tipos de criptomonedas para hacer el ejercicio más práctico.

In [45]:
new_data = data[data['Type'].isin([2, 4, 5])][[
    'p1_d',
    'p2_d',
    'min_d',
    'Avg_bpp',
    'p1_ip',
    'p2_ip',
    'p3_ip',
    'p2_ib',
    'Type']]

In [46]:
new_data.head()

,p1_d,p2_d,min_d,Avg_bpp,p1_ip,p2_ip,p3_ip,p2_ib,Type
1181,114.79300,180.04350,114.73600,394,10.00000,12.50000,15.00000,"3,820.00000",2
1182,196.18225,196.18950,196.17500,469,10.50000,11.00000,11.50000,"5,159.00000",2
1183,244.14325,244.14350,244.14300,425,16.00000,16.00000,16.00000,"6,807.50000",2
1184,244.04600,244.04600,244.04600,419,16.00000,16.00000,16.00000,"6,707.00000",2
1185,246.79500,246.79500,246.79500,376,17.75000,18.50000,19.25000,"6,959.50000",2


# **Data Split**
---


In [47]:
X=new_data.iloc[:,:-1]
y=new_data.iloc[:,-1]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=True,
    random_state=random_seed
)

In [49]:
y_test.value_counts()

,count
Type,
5,54
2,51
4,37


# **Model**

---



A DataFrame is defined to compare the predictions.

°°°°

Se define un DataFrame para comparar las predicciones.

In [50]:
predictions = pd.DataFrame(columns=['params_ovo', 'multiple_model', 'OneVsOneClassifier'])

**params_ovo**

An SVC (Support Vector Classifier) is implemented with the OVO (One-vs-One) strategy parameter.

°°°°

Se implementa un SVC (clasificador de vectores de soporte) con el parámetro de estrategia OVO (uno contra uno).

In [51]:
model_svc = SVC(decision_function_shape='ovo', random_state=random_seed)
model_svc.fit(X_train, y_train)

SVC(decision_function_shape='ovo', random_state=73)

In [52]:
y_predict = model_svc.predict(X_test)
predictions['params_ovo'] = y_predict

**multiple_model**





To simulate the One-vs-One strategy, a model was implemented for each pair of labels.

°°°°

Para simular la estrategia Uno vs Uno, se implementó un modelo para cada par de etiquetas.

In [53]:
(3*(3-1))/2

3.0

In [54]:
X_new_train = pd.concat([X_train, y_train], axis=1)

**multiple model**

Data for cryptocurrency two and four

In [55]:
data_bitcoin_monero_X = X_new_train[X_new_train['Type'].isin([2, 4])]
data_bitcoin_monero_y = data_bitcoin_monero_X.iloc[:,-1]
data_bitcoin_monero_X = data_bitcoin_monero_X.iloc[:, :-1]

Data for cryptocurrency two and five

In [56]:
data_bitcoin_litecoin_X = X_new_train[X_new_train['Type'].isin([2, 5])]
data_bitcoin_litecoin_y = data_bitcoin_litecoin_X.iloc[:,-1]
data_bitcoin_litecoin_X = data_bitcoin_litecoin_X.iloc[:, :-1]

Data for cryptocurrency four and five

In [57]:
data_monero_litecoin_X = X_new_train[X_new_train['Type'].isin([4, 5])]
data_monero_litecoin_y = data_monero_litecoin_X.iloc[:,-1]
data_monero_litecoin_X = data_monero_litecoin_X.iloc[:, :-1]

Model for cryptocurrency bitcoin and monero

In [58]:
model_svc_bitcoin_monero = SVC(random_state=random_seed)
model_svc_bitcoin_monero.fit(data_bitcoin_monero_X, data_bitcoin_monero_y)
y_predict_bitcoin_monero = model_svc_bitcoin_monero.predict(X_test)
y_predict_bitcoin_monero

array([4, 2, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 4, 4, 2, 2,
       4, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 4, 4, 2, 2, 4, 4, 4, 2, 4,
       2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 4, 2,
       2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2,
       2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 4, 4, 4, 2, 4, 2, 2, 4, 2,
       2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 2, 2, 2, 4])

Model for cryptocurrency bitcoin and litecoin

In [59]:
model_svc_bitcoin_litecoin = SVC(random_state=random_seed)
model_svc_bitcoin_litecoin.fit(data_bitcoin_litecoin_X, data_bitcoin_litecoin_y)
y_predict_bitcoin_litecoin = model_svc_bitcoin_litecoin.predict(X_test)
y_predict_bitcoin_litecoin

array([2, 2, 2, 5, 2, 2, 5, 2, 5, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2,
       2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 5, 5, 2, 2, 5, 5, 2, 2, 2, 5, 2,
       5, 2, 5, 5, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2,
       2, 2, 2, 5, 2, 2, 2, 2, 5, 2, 5, 2, 5, 2, 2, 5, 2, 2, 5, 2, 2, 5,
       2, 2, 5, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2,
       2, 2, 5, 5, 2, 5, 2, 2, 2, 2, 5, 5, 2, 2, 2, 2, 2, 2, 2, 5, 2, 5,
       5, 2, 5, 2, 2, 2, 2, 2, 2, 2])

Model for cryptocurrency monero and litecoin

In [60]:
model_svc_monero_litecoin = SVC(random_state=random_seed)
model_svc_monero_litecoin.fit(data_monero_litecoin_X, data_monero_litecoin_y)
y_predict_monero_litecoin = model_svc_monero_litecoin.predict(X_test)
y_predict_monero_litecoin

array([4, 5, 5, 5, 5, 4, 5, 4, 5, 4, 5, 4, 4, 5, 4, 5, 5, 5, 4, 4, 5, 5,
       4, 4, 4, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 4, 4, 5, 5, 4, 4, 4, 5, 4,
       5, 5, 5, 5, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4, 4, 5, 5, 5, 4, 5, 4, 5,
       5, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 4, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5,
       5, 4, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 5, 4, 4, 4, 5, 4, 5, 4, 4, 5,
       5, 4, 5, 5, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 4, 5, 4, 5, 4, 5, 5, 5,
       5, 5, 5, 4, 5, 5, 5, 5, 5, 4])

The predictions from each binary model are concatenated to extract the final prediction.

°°°°

Las predicciones de cada modelo binario se concatenan para extraer la predicción final.

In [61]:
voting = np.stack(
    [
        y_predict_bitcoin_monero,
        y_predict_bitcoin_litecoin,
        y_predict_monero_litecoin
    ],
    axis=1
)

predictions['multiple_model'] = pd.DataFrame(voting).apply(lambda x: x.value_counts().index[0], axis=1)

**OneVsOneClassifier**

The 'OneVsOneClassifier' function from scikit-learn is used to implement the strategy automatically. This function allows developing the strategy mentioned in this notebook for any model.

°°°°

La función 'OneVsOneClassifier' de scikit-learn se utiliza para implementar la estrategia de forma automática. Esta función permite desarrollar la estrategia mencionada en este cuaderno para cualquier modelo.

In [62]:
model_svc = SVC(random_state=random_seed)
one_vs_one = OneVsOneClassifier(model_svc)
one_vs_one.fit(X_train, y_train)
y_predict = one_vs_one.predict(X_test)
predictions['OneVsOneClassifier'] = y_predict

The test labels are appended.

°°°°

se anexa las etiquetas de prueba

In [63]:
predictions['labels_true'] = y_test.reset_index(drop=True)

In [71]:
predictions

,params_ovo,multiple_model,OneVsOneClassifier,labels_true
0,4,4,4,4
1,2,2,2,5
2,2,2,2,5
3,5,5,5,5
4,2,2,2,2
...,...,...,...,...
137,2,2,2,2
138,2,2,2,2
139,2,2,2,2
140,2,2,2,2


**Info**
---
**@By:** Steven Bernal

**@Nickname:** Kaiziferr

**@Git:** https://github.com/Kaiziferr